In [16]:

import glob
import os
from os import listdir,mkdir,rmdir
from os.path import join,isdir,isfile
from PIL import Image

import cv2
import numpy as np
from skimage import exposure

%matplotlib inline
import matplotlib.pyplot as plt

In [17]:
plt.rcParams['figure.figsize'] = 16,9
plt.rcParams.update({'font.size': 22})

In [18]:
img_size = 512

In [19]:
def normalize_img(img):
    img -= img.min()
    img /= (img.max() + 1e-6)
    return img

In [20]:
def prepare_array(path_img,path_seg):
    global img_size
    img = np.array(Image.open(path_img)).astype(np.float32)
    seg = np.array(Image.open(path_seg)).astype(np.float32)
    
    img = normalize_img(img)
    #seg = (seg > 0)
    img = exposure.equalize_adapthist(img, clip_limit=0.03)
    
    img = cv2.resize(img,(img_size,img_size))
    seg = cv2.resize(seg,(img_size,img_size))
    seg = (seg > 0)
    
    img = (255*img).astype(np.uint8)
    seg = (255*seg).astype(np.uint8)
    
    return img,seg

In [21]:
path_data = "/media/darvin/Data/cellSegmentation"
path_data_tr = join(path_data, "rawDataForTraining")
name_val = "Mask_20130704_CeruUTR_10"
path_data_te = join(path_data, "rawDataForValidation")

path_save = "/media/darvin/Data/cellSegmentation"
path_save_tr = join(path_save, "train")
path_save_va = join(path_save, "val")
path_save_te = join(path_save, "test")

if not isdir(path_save):
    mkdir(path_save)
for path_to_make in [path_save_tr, path_save_va, path_save_te]:
    if not isdir(path_to_make):
        mkdir(path_to_make)
    if not isdir(join(path_to_make, "img")):
        mkdir(join(path_to_make, "img"))
    if not isdir(join(path_to_make, "seg")):
        mkdir(join(path_to_make, "seg"))

In [22]:
for ii,name_pt in enumerate(sorted(listdir(path_data_tr))):
    if name_pt == name_val:
        path_to_save = path_save_va
    else:
        path_to_save = path_save_tr
    
    path_data_pt = join(path_data_tr, name_pt)
    for name_img in sorted(listdir(path_data_pt)):
        if name_img[:3] != "DIC":
            continue
        name_seg = "Nul" + name_img[3:]
        path_img = join(path_data_pt,name_img)
        path_seg = join(path_data_pt,name_seg)
        
        img,seg = prepare_array(path_img,path_seg)
        
        cv2.imwrite(join(path_to_save,"img",name_pt + '.' + name_img[4:-4] + '.png'),img)
        cv2.imwrite(join(path_to_save,"seg",name_pt + '.' + name_img[4:-4] + '.png'),seg)

In [23]:
for ii,name_pt in enumerate(sorted(listdir(path_data_te))):
    path_to_save = path_save_te
    
    path_data_pt = join(path_data_te, name_pt)
    for name_img in sorted(listdir(path_data_pt)):
        if name_img[:3] != "DIC":
            continue
        name_seg = "Nul" + name_img[3:]
        path_img = join(path_data_pt,name_img)
        path_seg = join(path_data_pt,name_seg)
        
        img,seg = prepare_array(path_img,path_seg)
        
        cv2.imwrite(join(path_to_save,"img",name_pt + '.' + name_img[4:-4] + '.png'),img)
        cv2.imwrite(join(path_to_save,"seg",name_pt + '.' + name_img[4:-4] + '.png'),seg)